In [3]:
import mesa
import random
import math
from mesa import Agent, Model
from mesa.time import RandomActivation
from scipy import stats
import numpy as np
import cvxpy as cp

How does arbitrage work?

Note that $\Delta_a^\star = \max((R_a - \sqrt{k/(1-\gamma )m_{A \to B}}), 0)$

where $m_{A\to B} = A/B = .5$


Example Let us have a Pool $(R_a, R_b)$ where $R_b/R_a = .5$ and $R_a/R_b = 2$. (ie 2 Token A = 1 Token B)

Suppose the pool started as $(12, 6)$ and trades to be $(8,9)$ where $k = 72$ and $\gamma = 0$

We calculate $\Delta_a^\star = \max(8-\sqrt{72/.5}, 0) = \max(8-12, 0) = 0$

We calculate $\Delta_b^\star = \max(9-\sqrt{72/2}, 0) = \max(9-6, 0) = 3$.

Therefore $\Delta_b^\star$ is a valid arbitrage position.


Let us then run this arbitrauge. Suppose we use $\Delta_b^\star$ to calculate an amount to borrow. We borrow that amount, make the trade and pay ourselves back taking profit being AmountB - BorrowA.

We calculate BorrowA = $k/(R_b - \Delta_b^\star) - R_a$ 

We then SwapAtoB(BorrowA) and get newB

We then convert newB to asset A and subtract the amount we borrowed to get profit

In the case of our values above we get that BorrowA $= (72/9-3) - 8 = 4$

We then SwapAtoB(4) = 3 Token B

We see that 3 TokenB = 6 TokenA. 

Profit = 6 - 4 = 2

Thus we make a profit of $2$ token A under no fee and no interest rate





In [53]:

#Update Market Price
# Market Price A
# Market price B
# Finish 
#Noise Trader swaps AtoB or BtoA at random based off of flip of a hand
# Arbitrauger (convert problem to optimizations (might be able to grab it from a paper))
#


class NoiseTraders(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealthA = 100.0
        self.wealthB = 100.0
        self.wealthC = 0.0

    def step(self):
        
        print("Noise Trader Step")
        
        constant = 10
        A = self.model.NormalX() * constant
        B = self.model.NormalX() * constant
        
        AorB = random.randint(0,1)
            #draw for A or B randNorm(0,1) * C -> x
            # OR
            # Trade no matter what, randNorm() * C
            #TODO? A/B < .98 RefB -> SwapBToA
        if AorB == 0:
            if self.wealthA >= A:
                BOut = self.model.AMM_Model.swapAtoB(A)
                self.wealthA -= A
                self.wealthB += BOut
                print("AtoB swap: ", BOut)
        else:
            if self.wealthB >= B:
                AOut = self.model.AMM_Model.swapBtoA(B)
                #print("SelfReferencePrice", self.model.ReferencePriceA)
                self.wealthB -= B
                self.wealthB += AOut
                print("BtoA swap: ", AOut)
        
        #if self.model.AMM_Model.ReserveB / self.model.AMM_Model.ReserveA < (.98) *self.model.ReferencePriceA:
            #AorB = random.randint(0,1)
            #draw for A or B randNorm(0,1) * C -> x
            # OR
            # Trade no matter what, randNorm() * C
            #TODO? A/B < .98 RefB -> SwapBToA
            #if AorB == 0:
                #if self.wealthA >= 10:
                    #BOut = self.model.AMM_Model.swapAtoB(10)
                    #self.wealthA -= 10
                    #self.wealthB += BOut
                    #print("AtoB swap: ", BOut)
           # else:
                #if self.wealthB >= 10:
                    #AOut = self.model.AMM_Model.swapBtoA(10)
                    #print("SelfReferencePrice", self.model.ReferencePriceA)
                    #self.wealthB -= 10
                    #self.wealthB += AOut
                    #print("BtoA swap: ", AOut)
        #else:
            #print("No swap made by noise trader")
         

class InitialLiqudityProviders(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.totalInitialWealthB = 1000
        self.wealthA = self.model.w_data[0][0] * self.totalInitialWealthB * self.model.ReferencePriceA
        self.wealthB = self.model.w_data[0][1] * self.totalInitialWealthB
        #print("SelfReferencePrice", self.model.ReferencePriceA)
        self.wealthC = 0.0
        self.Intialize_Pool()
        
    def step(self):
        x =0
        
    def Intialize_Pool(self):
        print(self.model.w_data[0])
        
        amountBToDeposit = (self.model.w_data[0][2] * self.totalInitialWealthB)/2
        self.wealthC = self.model.AMM_Model.addDeposit(amountBToDeposit * self.model.ReferencePriceA, amountBToDeposit)
        #self.wealthA -= amountBToDeposit/self.model.ReferencePriceA
        #self.wealthB -= amountB
        print("wealth A", self.wealthA)
        print("wealth B", self.wealthB)
        print("wealth C", self.wealthC) 
        print("\n")
        
            

class Arbitraugers(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealthA = 100.0
        self.wealthB = 100.0
        self.wealthC = 0.0

    def step(self):
        print("Arbitrauger Step")
        
        
        OptimalA = max(self.model.AMM_Model.ReserveA - math.sqrt(self.model.AMM_Model.K/ 
                                ((1-self.model.AMM_Model.Fee)* self.model.ReferencePriceB)), 0 )
        
        
        #Searches for Arb from B to A
        if OptimalA > 0:
            #TODO Markdown Arbitrage Logic 
            ## TODO add lending rates here
            
            BorrowB = self.model.AMM_Model.K/(self.model.AMM_Model.ReserveA - OptimalA) - self.model.AMM_Model.ReserveB
           
            NewA = self.model.AMM_Model.K/(self.model.AMM_Model.ReserveB - BorrowB) - self.model.AMM_Model.ReserveA
        
            ## The Amount to borrow
            print("Borrow Amount", BorrowB)
            ## referencePriceA * OptimalA - BorrowB should be profit
            
            print("New B", NewA)
            
            print("NetValue", NewA * self.model.ReferencePriceB)
            
            if NewA * self.model.ReferencePriceB - BorrowB > 0:
                ReturnedA = self.model.AMM_Model.swapBtoA(BorrowB)
                #pay back the loan of B
                profit = ReturnedA * self.model.ReferencePriceB - BorrowB
                self.wealthB += profit
        
                print("Arbitrauger made profit: ", profit, " borrowing B and swapping A")
                print("\n")
            
           
        
        OptimalB = max(self.model.AMM_Model.ReserveB - math.sqrt(self.model.AMM_Model.K/ ((1-self.model.AMM_Model.Fee)* self.model.ReferencePriceA)), 0 )
        
        #Searches for Arb from A to B
        if OptimalB > 0:
            BorrowA = self.model.AMM_Model.K/(self.model.AMM_Model.ReserveB - OptimalB) - self.model.AMM_Model.ReserveA
            print("Borrow Amount", BorrowA)
            NewB = self.model.AMM_Model.K/(self.model.AMM_Model.ReserveA - BorrowA) - self.model.AMM_Model.ReserveB
            ## The Amount to borrow
            ## TODO add lending rates here
            ## referencePriceA * OptimalA - BorrowB should be profit
            
            print("NetValue", NewB * self.model.ReferencePriceA)
            
            if NewB * self.model.ReferencePriceA - BorrowA > 0:
                ReturnedB = self.model.AMM_Model.swapAtoB(BorrowA)
                #pay back the loan of B
                profit = ReturnedB * self.model.ReferencePriceA - BorrowA
                self.wealthA += profit
        
                print("Arbitrauger made profit: ", profit, " borrowing A and swapping B")
                print("\n")
            
            
            
            
           
            
        
            
        
        

class Model(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, nInitialLP, nNoiseTrader, nArbitraugers):
        ##Reference Price of TokenA
        self.AMM_Model = AMM()
        self.X = self.NormalX()
        self.sigma = 0.04
        self.mu = 0
        self.ReferencePriceA = random.uniform(1, 100)
        self.ReferencePriceB = 1/self.ReferencePriceA
        
        self.SAMPLES = 1
        self.risk_data = np.zeros(self.SAMPLES)
        self.ret_data = np.zeros(self.SAMPLES)
        self.w_data = np.zeros((self.SAMPLES, 3))
        self.calculate_portfolio() 
        self.schedule = mesa.time.RandomActivation(self)
        # Create Initial Liqiduity Providers agents
        for i in range(nInitialLP):
            a = InitialLiqudityProviders(i, self)
            self.schedule.add(a)

        for i in range (nInitialLP, nInitialLP+nNoiseTrader):
            a = NoiseTraders(i, self)
            self.schedule.add(a)

        for i in range (nInitialLP+nNoiseTrader, nInitialLP+nNoiseTrader+nArbitraugers):
            a = Arbitraugers(i, self)
            self.schedule.add(a)
        
    def calculate_portfolio(self):
        mu = np.array([self.mu, -1 * self.mu, .5 * self.mu])
        
        ## aa ab ac
        ## ba bb bc
        ## ca cb cc
        #Sigma = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
        Sigma = np.array([[(math.exp(2 * self.mu + self.sigma**2) * (math.exp(self.sigma**2)-1)), (math.exp(self.sigma**2) * (math.exp(-1 * self.sigma**2) -1)), (math.exp(1.5 * self.mu  + .5 *(1.25 * self.sigma**2)) * (math.exp(1.25 * self.sigma**2) -1 ))],
                        [(math.exp(self.sigma**2) * (math.exp(-1 * self.sigma**2) -1)), (math.exp(-2 * self.mu + self.sigma**2) * (math.exp(self.sigma**2) -1)), (math.exp(-.5 * self.mu + .5 * (1.25 * self.sigma**2)) * (math.exp(-.5 * self.sigma**2) -1))],
                        [(math.exp(1.5 * self.mu  + .5 *(1.25 * self.sigma**2)) * (math.exp(1.25 * self.sigma**2) -1 )), (math.exp(-.5 * self.mu + .5 *(1.25 * self.sigma**2)) * (math.exp(-.5 * self.sigma**2) -1)), (math.exp(self.mu + .25 * self.sigma**2) * (math.exp(.25 * self.sigma**2) -1)) ]])
        Sigma = Sigma.T.dot(Sigma)
        w = cp.Variable(3)
        print(w)
        gamma = cp.Parameter(nonneg=True)
        ret = mu.T @ w
        risk = cp.quad_form(w, Sigma)
        prob = cp.Problem(cp.Maximize(ret - gamma * risk), [cp.sum(w) == 1, w >= 0])
        gamma_vals = np.logspace(1, 2, num=self.SAMPLES)
       
        
        
        for i in range(self.SAMPLES):
            gamma.value = gamma_vals[i]
            prob.solve()
            self.risk_data[i] = cp.sqrt(risk).value
            self.ret_data[i] = ret.value
            self.w_data[i] = w.value
            prob.solve()
            #print(self.w_data[i])
        
    def step(self):
        """Advance the model by one step."""
        self.schedule.step()

    def NormalX(self):
        low = 0
        high = 1
        mean = 0.55
        stddev = 0.2

        return stats.truncnorm.rvs(low, high,
                             loc = mean, scale = stddev,
                             size = 1)
    
    def UpdateReferencePriceA(self):
        self.ReferencePriceA = self.ReferencePriceA * math.exp(self.sigma * self.X + self.mu)
        self.ReferencePriceB = 1/self.ReferencePriceA
        
        
    def UpdateReferencePriceB(self):
        
        self.ReferencePriceB = (1/self.ReferencePriceA) * math.exp(self.sigma * (-1) * self.X + self.mu)
        self.ReferencePriceA = 1/self.ReferencePriceB

class AMM():
    def __init__(self):
        self.Fee = 0.03
        self.K = 0
        self.totalShares = 0.0
        self.ReserveA = 0.0
        self.ReserveB = 0.0

    def addDeposit(self, NewReserveA, NewReserveB):
        if self.totalShares == 0: 
            self.ReserveA = NewReserveA
            self.ReserveB = NewReserveB

            self.K = self.ReserveA * self.ReserveB
            self.totalShares = self.K
            return self.totalShares
        else: 
            
            sharesMinted = self.totalShares * (NewReserveB / self.ReserveB)
            self.ReserveA += NewReserveA
            self.ReserveB += NewReserveB

            self.K = self.ReserveA * self.ReserveB
            self.totalShares += sharesMinted
            return sharesMinted
        
    def swapAtoB(self, NewReserveA):
        NewReserveB = ((-1) * self.K)/(self.ReserveA + ((1-self.Fee) * NewReserveA)) +self.ReserveB
        self.ReserveA += NewReserveA
        self.ReserveB -= NewReserveB
        print("Old k", self.K)
        self.K = self.ReserveA * self.ReserveB
        print("New K", self.K)
        
        return NewReserveB
    
    def swapBtoA(self, NewReserveB):
        NewReserveA = ((-1) * self.K)/(self.ReserveB + ((1-self.Fee) * NewReserveB)) +self.ReserveA
        #print("New Reserve B: ", NewReserveB)
        #print("ReserveA", self.ReserveA)
        
        #print("ReserveB", self.ReserveB)
        #print("New Reserve A Swap B to A", NewReserveA)
        self.ReserveA -= NewReserveA
        self.ReserveB += NewReserveB
        print("Old k", self.K)
        self.K = self.ReserveA * self.ReserveB
        print("New K", self.K)
        
        return NewReserveA
       


In [54]:
model = Model(10, 2, 2)
print("Initial ReserveA", model.AMM_Model.ReserveA)
print("Initial ReserveB", model.AMM_Model.ReserveB)
print("\n")
for x in range(0, 10):
    print("RefB", model.ReferencePriceB)
    print("RefA", model.ReferencePriceA)
    model.UpdateReferencePriceB()
    
    model.step()
print("\n")
print("New ReserveA", model.AMM_Model.ReserveA)
print("New ReserveB", model.AMM_Model.ReserveB)

    

var1387
[0.21577251 0.50195659 0.2822709 ]
wealth A 18426.536822989998
wealth B 501.9565875225371
wealth C 1701060.8012442745


[0.21577251 0.50195659 0.2822709 ]
wealth A 18426.536822989998
wealth B 501.9565875225371
wealth C 1701060.8012442745


[0.21577251 0.50195659 0.2822709 ]
wealth A 18426.536822989998
wealth B 501.9565875225371
wealth C 1701060.8012442745


[0.21577251 0.50195659 0.2822709 ]
wealth A 18426.536822989998
wealth B 501.9565875225371
wealth C 1701060.8012442742


[0.21577251 0.50195659 0.2822709 ]
wealth A 18426.536822989998
wealth B 501.9565875225371
wealth C 1701060.8012442742


[0.21577251 0.50195659 0.2822709 ]
wealth A 18426.536822989998
wealth B 501.9565875225371
wealth C 1701060.8012442742


[0.21577251 0.50195659 0.2822709 ]
wealth A 18426.536822989998
wealth B 501.9565875225371
wealth C 1701060.8012442742


[0.21577251 0.50195659 0.2822709 ]
wealth A 18426.536822989998
wealth B 501.9565875225371
wealth C 1701060.8012442742


[0.21577251 0.50195659 0.2822709

In [31]:
logGrossReturnsReserveAlist = []
logGrossReturnsReserveBlist = []
logGrossReturnsReserveClist = []

for x in range(0, 1):
    model = Model(5, 2, 2)
    model.UpdateReferencePriceA()
    model.UpdateReferencePriceB()
    model.calculate_portfolio()
    model.step()
    #print("Log gross returns: R_A: ", model.sigma * model.X + model.mu)
    logGrossReturnsReserveAlist.append(model.sigma * model.X + model.mu)
    #print("Log gross returns: R_B: ", (-1) * model.sigma * model.X - model.mu)
    logGrossReturnsReserveBlist.append((-1) * model.sigma * model.X - model.mu)
    #print("Log gross returns: R_C: ", .5 * (model.sigma * model.X + model.mu))
    logGrossReturnsReserveClist.append(.5 * (model.sigma * model.X + model.mu))
    
print("Reserve A")
print("ReserveA Log Gross Returns Average: ", np.average(logGrossReturnsReserveAlist))
print("ReserveA Log Gross Returns Variance: ",np.var(logGrossReturnsReserveAlist))    
print("ReserveA Log Gross Returns Standard Deviation: ", np.std(logGrossReturnsReserveAlist))

print("Reserve B")
print("ReserveB Log Gross Returns Average: ", np.average(logGrossReturnsReserveBlist))
print("ReserveB Log Gross Returns Variance: ",np.var(logGrossReturnsReserveBlist))    
print("ReserveB Log Gross Returns Standard Deviation: ", np.std(logGrossReturnsReserveBlist))

print("Liqudity Shares")
print("Liqudity Shares Log Gross Returns Average: ", np.average(logGrossReturnsReserveClist))
print("Liqudity Shares Gross Returns Variance: ",np.var(logGrossReturnsReserveClist))    
print("Liqudity Shares Gross Returns Standard Deviation: ", np.std(logGrossReturnsReserveClist))

var799
SelfReferencePrice 33.571894602769234
[ 1.00000000e+00  1.11025849e-22 -5.55095035e-23]
wealth A 33571.89460276924
wealth B 1.1102584937153844e-19
wealth C 2.586131152748312e-38
SelfReferencePrice 33.571894602769234
[ 1.00000000e+00  1.11025849e-22 -5.55095035e-23]
wealth A 33571.89460276924
wealth B 1.1102584937153844e-19
wealth C 2.586131152748312e-38
SelfReferencePrice 33.571894602769234
[ 1.00000000e+00  1.11025849e-22 -5.55095035e-23]
wealth A 33571.89460276924
wealth B 1.1102584937153844e-19
wealth C 2.586131152748312e-38
SelfReferencePrice 33.571894602769234
[ 1.00000000e+00  1.11025849e-22 -5.55095035e-23]
wealth A 33571.89460276924
wealth B 1.1102584937153844e-19
wealth C 2.586131152748312e-38
SelfReferencePrice 33.571894602769234
[ 1.00000000e+00  1.11025849e-22 -5.55095035e-23]
wealth A 33571.89460276924
wealth B 1.1102584937153844e-19
wealth C 2.586131152748312e-38
var841
0.02978682054832596
35.183095815590065
Old k 6.465327881870779e-37
New K 0.0
BtoA swap:  -4.6588

/var/folders/sl/60nq5_6d7d50jlb5hk9ys0cr0000gn/T/ipykernel_50818/3080707948.py:122: RuntimeWarning: invalid value encountered in double_scalars
  BorrowA = self.model.AMM_Model.K/(self.model.AMM_Model.ReserveB - OptimalB) - self.model.AMM_Model.ReserveA
/var/folders/sl/60nq5_6d7d50jlb5hk9ys0cr0000gn/T/ipykernel_50818/3080707948.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  print(self.model.AMM_Model.ReserveB / self.model.AMM_Model.ReserveA)
/var/folders/sl/60nq5_6d7d50jlb5hk9ys0cr0000gn/T/ipykernel_50818/3080707948.py:23: RuntimeWarning: divide by zero encountered in double_scalars
  if self.model.AMM_Model.ReserveB / self.model.AMM_Model.ReserveA < (.98) *self.model.ReferencePriceA:


In [69]:
model = Model(5, 0, 0)


var845266
SelfReferencePrice 70.22830584951356
[0.21577251 0.50195659 0.2822709 ]
wealth A 15153.33784128745
wealth B 501.9565875225371
wealth C 352465.6021788797
SelfReferencePrice 70.22830584951356
[0.21577251 0.50195659 0.2822709 ]
wealth A 15153.33784128745
wealth B 501.9565875225371
wealth C 352465.6021788797
SelfReferencePrice 70.22830584951356
[0.21577251 0.50195659 0.2822709 ]
wealth A 15153.33784128745
wealth B 501.9565875225371
wealth C 352465.6021788797
SelfReferencePrice 70.22830584951356
[0.21577251 0.50195659 0.2822709 ]
wealth A 15153.33784128745
wealth B 501.9565875225371
wealth C 352465.60217887966
SelfReferencePrice 70.22830584951356
[0.21577251 0.50195659 0.2822709 ]
wealth A 15153.33784128745
wealth B 501.9565875225371
wealth C 352465.60217887966
